In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
datadir = r"/content/gdrive/MyDrive/bosch_data"  

In [ ]:
data_files = ['train_numeric', 'train_date', 'train_categorical']
def explore_data_size(files):
  # explore the size (rows, cols) of each file   
  stats = []
  for file_name in data_files:
      cols = pd.read_csv(os.path.join(datadir,file_name+'.csv.zip'), nrows=1)
      rows = pd.read_csv(os.path.join(datadir,file_name+'.csv.zip'), usecols=["Id"])
      stats.append({'File': file_name, 'Rows': rows.shape[0], 'Columns': cols.shape[1]})
  # convert the result into a DataFrame so we can do plotting.
  df = pd.DataFrame(stats, index=["File"], columns=["Rows", "Columns"])
  return df

In [ ]:
def get_station_failure(date, res):
  df = date.copy()
  df.columns = pd.MultiIndex.from_tuples([tuple([int(a[1:]) for a in x[3:].split('_')]) for x in df.columns], names=['station', 'feature'])
  df = df.groupby(level=0,axis=1).min()
  stations = df.columns
  date_res = pd.concat([df,res],axis=1)
  fail_counts_per_station = date_res.loc[date_res.Response==1,stations].notna().sum(axis=0)
  success_counts_per_station = date_res.loc[date_res.Response==0,stations].notna().sum(axis=0)
  fail_rate_per_station = date_res.loc[date_res.Response==1,stations].notna().sum(axis=0)/date_res[stations].notna().sum(axis=0)

In [ ]:
def get_line_failure(date, res):
  df = date.copy()
  tuples = [tuple(int(a[1:]) for a in (x.split('_')[0],x.split('_')[2])) for x in df.columns]
  new_columns = pd.MultiIndex.from_tuples(tuples, names=['line', 'feature'])
  df = df.groupby(level=0,axis=1).min()
  lines = df.columns
  date_res = pd.concat([df,res],axis=1)
  fail_counts_per_line = date_res.loc[date_res.Response==1,lines].notna().sum(axis=0)
  success_counts_per_line = date_res.loc[date_res.Response==0,lines].notna().sum(axis=0)
  fail_rate_per_line = date_res.loc[date_res.Response==1,stations].notna().sum(axis=0)/date_res[lines].notna().sum(axis=0)

In [ ]:
def get_line_station():
  date = splits_cols('train_date',D=True)
  line_station = {}
  for i in date.Line.unique():
    line_station[i] = date.loc[date.Line==i, 'Station'].unique()
  return line_station

In [ ]:
def explore_Tx_value(df):
  tem = df.fillna(-99).values 
  cate_uniques = np.unique(tem[tem!=-99]) 
  return cate_uniques

In [ ]:
def explore_sparse_rate(df,name):
  sparse_rate = df.isnull().sum(axis = 1)/df.shape[1]
  print("the average sparse rate of {} is: {:.2%}".format(name, sparse_rate.mean()))
  return sparse_rate

In [ ]:
def splits_cols(file,D=False,r=False):
  df = pd.read_csv(os.path.join(datadir,file+'.csv.zip'), index_col=0, compression='zip',nrows=0)
  if r:
    df.drop('Response',axis=1,inplace=True)
  splits = [tuple([int(a[1:])  for a in x.split('_')]) for x in df.columns]
  a = pd.DataFrame(splits,columns=['Line','Station','Feature'])
  if D:
    a['Feature'] = a['Feature'] - 1
  
  return a

In [ ]:
def line_station_structure():
  date = splits_cols('train_date',D=True)
  line_fcounts = date.groupby('Line')['Feature'].count()
  station_fcounts = date.groupby('Station')['Feature'].count()
  line_station = pd.concat([date.Station,date.Line],axis=1).drop_duplicates(keep='first',inplace=True).reset_index(drop=True,inplace=True)
  line_stcounts = line_station.groupby('Line').count()
  #plot: line-fcount,station-fount(line color),line-stationcount
  #plot: station-feature scatter with line color, num and cat color

In [ ]:
def path_network(df,res):
  df = df.groupby(level=0,axis=1).min()          # date.min(axis=1, level=0)
  stations = df.columns.tolist()
  df['p'] = res.Response.apply(lambda x: 'p' if x==1 else np.nan)
  df['n'] = res.Response.apply(lambda x: 'n' if x==0 else np.nan)
  pos_idx = df[df.p=='p'].index   # 故障产品索引号
  neg_idx = df[df.n=='n'].index    # 正常产品 索引号
  stations.extend(['n', 'p'])
  # station names
  save_pickle(stations, 'stations.pickle')
  # -------------station node status----------------------------------
  node_weights = df[df.p=='p'].notna().sum(axis=0)/df.notna().sum(axis=0).to_dict()
  #path_time = df.apply(lambda x: x.dropna().tolist(), axis=1)
  total_samples = df.notna().sum(axis=0).to_dict()     # 通过每一个站点的数量
  total_pos = df[df.p=='p'].notna().sum(axis=0).to_dict()
  station_stat = {k: [node_weights[k], total_pos[k], total_samples[k]/df.shape[0], total_samples[k]] for k in node_weights.keys()}
  station_stat = pd.DataFrame.from_dict(station_stat, orient='index')
  station_stat.columns = ['error_rate', 'error_count', 'sample_rate', 'sample_count']
  station_stat.index.name = 'station'
  save_pickle(station_stat, 'node_station_stat.pickle')
  del total_samples, total_pos, station_stat
  gc.collect()
  # -------------transition edge status------------------------------------
  path_station = df.apply(lambda x: x.dropna().index.tolist(), axis=1)   # 获取通过的站点号
  path_station_list = path_station.apply(lambda x: [(x[i], x[i+1]) for i in range(len(x)-1)]).values.tolist()
  path_station_list_pos = path_station.loc[pos_idx].apply(lambda x: [(x[i], x[i+1]) for i in range(len(x)-1)]).values.tolist()
  path_station_list = pd.Series([x for a in path_station_list for x in a])    # 将每行的传递tuple组分散到一起
  path_station_list_pos = pd.Series([x for a in path_station_list_pos for x in a])
  edges = path_station_list.value_counts().to_dict()
  edges_pos = path_station_list_pos.value_counts().to_dict()
  edges_pos_rate = {k: edges_pos[k]/edges[k] for k in edges_pos.keys()}
  # Save transition error statistics
  edges_stat = {k: [edges[k], edges_pos[k], edges_pos_rate[k]] for k in edges_pos.keys()}
  edges_stat = pd.DataFrame.from_dict(edges_stat, orient='index')
  edges_stat.columns = ['sample_count', 'error_count', 'error_rate']
  edges_stat.index.name = 'transition'
  save_pickle(edges_stat, 'edge_transition_stat.pickle')
  del path_station, path_station_list, path_station_list_pos, edges_stat
  gc.collect()
  # -------------station path flow----------------------------------
  station_path_flow = {}
  for i in path_station.index:
    station_path_flow[i] = '_'.join([str(x) for x in path_station[i]])
  station_path_flow = pd.DataFrame(station_path_flow)
  station_path_flow['response'] = station_path_flow[0].apply(lambda x: 0 if x.split('_')[-1]=='n' else 1)
  station_path_flow[0] = station_path_flow[0].apply(lambda x: x[:-2])
  station_path_flow.columns = ['flow', 'response']
  station_path_flow_stat = \
    station_path_flow.groupby('flow', sort=False).agg([np.sum, np.mean, 'count']).sort_values(
    by=[('response', 'sum'), ('response', 'mean'), ('response', 'count')], ascending=[False, False, False])
  save_pickle(station_path_flow_stat, 'station_flow_error_stat.pickle')

In [ ]:
date = pd.read_csv(os.path.join(datadir,'train_date'+'.csv.zip'), index_col=0, nrows=80000, compression='zip')
response = pd.read_csv(os.path.join(datadir,'train_numeric'+'.csv.zip'), index_col=0, usecols=[0,969], nrows=80000, compression='zip')
date_start = date.min(axis=1)
date_end = date.max(axis=1)
dt = pd.concat([date_start, date_end, date_end-date_start, response], axis=1)
dt.columns = ['date_start', 'date_end', 'date_duration', 'Response']

In [ ]:
x = dt.groupby('date_start').Response.agg(['mean', 'sum', 'count']).sort_values(['count'], ascending=False)
x.sort_index(inplace=True)
# interpolate time
from scipy.fftpack import fft, fftfreq
t = np.arange(x.index.min(), x.index.max(), 0.01) # time axis
x_counts = np.interp(t, x.index, x['count']) # interpolated part counts 
x_error_rate = np.interp(t, x.index, x['mean']) # interpolated part error rate
x_errors = np.interp(t, x.index, x['sum']) # interpolated part error
N = t.shape[0] # total number of time points
T = 0.005 # time interval
f = np.linspace(0.0, 1.0//(2.0*T), N//2) # frequency axis
y_counts = fft(x_counts) # fft of x_counts
y_errors = fft(x_errors) # fft of x_errors
y_error_rate = fft(x_error_rate) # fft of x_error_rate
plt.plot(f, 10*np.log10(2.0/N * np.abs(y_counts[:N//2]))) # plot frequency 
plt.axis([0, 0.33, -30.0, 1.0]) # zoom in
plt.plot([0.0595, 0.0595], [-30, 1])
plt.plot([0.119, 0.119], [-30, 1])

In [ ]:
def time_series_mod(date, period):  
  tmp = (date['date_start'] % (period / 10)) * 10
  date['date_start_mod1'] = tmp.values.astype(int)
  a = date.groupby('date_start_mod1').Response.agg(['mean', 'sum', 'count'])
  a.reset_index(inplace=True)
  a.sort_values('date_start_mod1', ascending=True)
  tmp = (date['date_end'] % (period / 10)) * 10
  date['date_end_mod1'] = tmp.values.astype(int)
  b = date.groupby('date_end_mod1').Response.agg(['mean', 'sum', 'count'])
  b.reset_index(inplace=True)
  b.sort_values('date_end_mod1', ascending=True)
  fig1 = plt.figure(figsize=(14, 7))
  plt.bar(a['date_start_mod1'], a['count'], alpha=0.7)
  plt.bar(b['date_end_mod1'], b['count'], alpha=0.7)
  # plt.plot(b['count']*10**-6)
  plt.show()
  fig2 = plt.figure(figsize=(14, 7))
  plt.bar(a['date_start_mod1'], a['mean'], alpha=0.7)
  plt.bar(b['date_end_mod1'], b['mean'], alpha=0.7)
  # plt.plot(b['count']*10**-6)
  plt.show()

In [ ]:
def calc_stat(x, feat):
  a = x.groupby(feat).Response.agg(['mean', 'sum', 'count']).sort_values(['count'], ascending=False)
  a['confidence_interval'] = 1.96 * np.sqrt(a['mean'] * (1-a['mean']) / a['count'])
  a['mean_low'] = a['mean'] - a['confidence_interval']
  a['mean_high'] = a['mean'] + a['confidence_interval']
  a.sort_values('count', ascending=False, inplace=True)
  return a

In [ ]:
date = pd.read_csv(os.path.join(datadir,'train_date'+'.csv.zip'), index_col=0, nrows=2000, compression='zip')
response = pd.read_csv(os.path.join(datadir,'train_numeric'+'.csv.zip'), index_col=0, use_columns=[0,969], compression='zip')
tuples = [tuple([int(a[1:]) for a in x[3:].split('_')]) for x in date.columns]
date.columns = pd.MultiIndex.from_tuples(tuples, names = ['station', 'feature'])
station_time = pd.DataFrame()
for c in date.columns.get_level_values(0).unique():
  station_time['S{}_start'.format(c)] = date.min(axis=1).values
  station_time['S{}_end'.format(c)] = date.max(axis=1).values
  station_time['S{}_duration'.format(c)] = station_time['S{}_end'.format(c)] - station_time['S{}_start'.format(c)] 
station_time = station_time.join(response)


In [ ]:
date = pd.read_csv(os.path.join(datadir,'train_date'+'.csv.zip'), index_col=0, nrows=2000, compression='zip')
response = pd.read_csv(os.path.join(datadir,'train_numeric'+'.csv.zip'), index_col=0, use_columns=[0,969], compression='zip')
tuples = [tuple(int(a[1:]) for a in (x.split('_')[0],x.split('_')[2])) for x in date.columns]
date.columns = pd.MultiIndex.from_tuples(tuples, names = ['line', 'feature'])
line_time = pd.DataFrame()
for c in date.columns.get_level_values(0).unique():
  line_time['L{}_start'.format(c)] = date.min(axis=1).values
  line_time['L{}_end'.format(c)] = date.max(axis=1).values
  line_time['L{}_duration'.format(c)] = line_time['L{}_end'.format(c)] - line_time['L{}_start'.format(c)] 
line_time = line_time.join(response)